In [141]:
%matplotlib inline
import sys
from pathlib import Path

import pandas as pd
import numpy as np

import lnpy
from lnpy.lnpicollectionutils import limited_collection


In [138]:
def analyze_path(path):
    """Analyze path of form lj.t07..."""
    system, temp, meta, volume, _, _, run, *ext = path.name.split('.')
    
    suffix =  ".".join(ext)

    return {
        'path': path,
        'system': f"{system}-{meta}",
        'temp': float(f"{temp[1]}.{temp[2:]}"),
        'volume': float(volume.lstrip('v')),
        'run': int(run.lstrip('r')),
        "typ": suffix.replace(".","_"),
        
    }

#function to tag 'LD' and 'HD' phases
def tag_phases2(x):
    """tag vapor (0) and liquid (1)"""
    if len(x) > 2:
        raise ValueError('bad tag function')
    argmax0 = np.array([xx.local_argmax()[0] for xx in x])
    return np.where(argmax0 <= x[0].shape[0] / 2, 0, 1)

def read_sat(path):
    """read sat data (needed to get lnz)"""
    return (
        pd.read_table(path, sep='\s+')
        .rename(columns=lambda x: (
            x
            .replace('*','')
            .replace('(1)','_0')
            .replace('(2)','_1')
            .replace('lnxi','lnz')
        ))
        .iloc[0]    
    )

def read_table(path, name):
    """Arbitrary table read"""
    return pd.read_table(path, sep='\s+', header=None, names=['n', name]).astype({'n': int}).set_index('n')[name]

In [139]:
paths = Path("../data/NIST_SRSW_Data/LJ_PURE/LJ_3sigma_LRC_TMMC/").glob('*')
# filter out lj_rc3....
paths = list(filter(lambda x: '_' not in x.name, paths))

In [142]:
# create list of files
df_list = pd.DataFrame(map(analyze_path, paths))

# pivot
df_p = df_list.pivot(columns=['typ'], index=['system','temp','volume','run'], values='path').reset_index()

In [335]:
def analyze_row(row):
    
    import lnpy.stability # noqa # fmt: off
    
    nmax=2
    
    energy = read_table(row.energy_dat, 'energy')
    lnpi = read_table(row.new_lnpi_dat, 'lnPi')
    energy = energy.loc[lnpi.index]
    
    
    sat = read_sat(row['sat_dat'])
    ref = lnpy.lnPiMasked.from_data(
        lnz=sat['lnz'],
        lnz_data=sat['lnz'],
        data=lnpi.values,
        state_kws={**row[['temp','volume','run','system']], 'beta': 1./row['temp']},
        extra_kws={'PE': energy.values}
    ).zeromax().pad()
    
    
    phase_creator= lnpy.segment.PhaseCreator(nmax=2, nmax_peak=4, ref=ref, 
                                         merge_kws=dict(efac=0.8), 
                                         tag_phases=tag_phases2
                                        )
    build_phases = phase_creator.build_phases_mu([None])
    
    with lnpy.set_options(joblib_use=False):

        c_course, c = limited_collection(
            build_phases, dlnz=0.1, digits=3, 
            offsets=[-15, +15], 
            collection_kws=dict(unstack=False),
            edge_distance_min=8, dens_min=1e-6
        )


        table = (
            c.xge.table(['ntot','betaOmega','PE', 'edge_distance'], default_keys=[], mask_stable=True, ref=ref)
            .reset_index('sample')
            .to_dataframe()
        )

        # build spinodal/binodal
        table_spin = None
        table_bino = None
        if nmax > 1:
            L_spin = {}
            first_spin = None
            for efac in np.arange(1.0, 0.0, -0.2):
                try:
                    spin, _ = c_course.spinodal(2, build_phases, inplace=False, 
                                                as_dict=False, efac=efac, build_kws=dict(efac=efac * 0.5))
                    L_spin[efac] = spin
                    if first_spin is None:
                        first_spin = spin
                except:
                    break        

            if len(L_spin) > 0:
                spins = lnpy.lnPiCollection.concat(L_spin, concat_kws=dict(names=['efac']), unstack=False)

                # calc this directly, as wlnPi has some oddities
                wmin = spins.wfe.dw.droplevel('phase_nebr').pipe(lambda x: x.groupby(x.index.names)).min()

                
                table_spin = (
                    spins.xge.table(['ntot','betaOmega','PE', 'edge_distance'], default_keys=[], ref=ref)#, dim_to_suffix='component')
                    .assign(dw=lambda x: ('sample', wmin.loc[x['sample']]))
                    .reset_index('sample')
                    .to_dataframe()
                )

            else:
                pass

            if table_spin is not None:
                try:
                    # grab first spinodal
                    spin = first_spin
                    bino, _ = c_course.binodal(2, build_phases, inplace=False, as_dict=False, spinodals=spin)
                    table_bino = (
                        bino.xge.table(['ntot','betaOmega','PE','edge_distance'], default_keys=[], ref=ref)
                        .reset_index('sample')
                        .to_dataframe()
                    )
                except:
                    pass
                
    
    sat =   sat.to_frame().transpose().assign(**row[['temp','volume','run','system']])

    
    return sat, table, table_spin, table_bino
                


## get data

In [336]:
from tqdm.notebook import tqdm 
from joblib import Parallel, delayed

In [354]:
seq = tqdm(df_p.iterrows(), total=len(df_p))

L = Parallel(n_jobs=-1)(delayed(analyze_row)(row) for i, row in seq)

  0%|          | 0/2004 [00:00<?, ?it/s]

In [355]:
Ls = {k : [] for k in ['sat','table','spin','bino']}
for sat, table, spin, bino in L:
    Ls['sat'].append(sat)
    Ls['table'].append(table)
    if spin is not None:
        Ls['spin'].append(spin)
    if bino is not None:
        Ls['bino'].append(bino)

In [560]:
dfs = {k: pd.concat(v, ignore_index=True) for k, v in Ls.items()}

In [562]:
# only select one spinodal
dfs['spin'] = dfs['spin'].query('efac==1.0')

In [563]:
# save raw data
p = Path('../data/NIST_SRSW_Data/LJ_PURE/LJ_3sigma_LRC_TMMC/'.replace('NIST_SRSW_Data','processed'))
p.mkdir(parents=True, exist_ok=True)

## save all (spin and bino)

In [582]:
for x in ['spin','bino']:
    dfs[x].to_csv(p / f"{x}-all.csv", index=False)

## Average files

In [568]:
import tools.pandas_utils
from importlib import reload
reload(tools.pandas_utils)

<module 'tools.pandas_utils' from '/Users/wpk/Documents/Research/CurrentResearch/Projects/NIST_SRSW_Data_Analysis/notebooks/tools/pandas_utils.py'>

In [569]:
def assign_values(df):
    return (
        df
        .eval(
            """
            dens = ntot / volume
            pres = -betaOmega * dens / beta
            pe_n = PE / ntot
            """
        )
    )

def get_stats(df, gcol):
    out = tools.pandas_utils.stats_agg_dataframe(df.drop('run', axis=1).pipe(assign_values), gcol=gcol, collapse=False, swaplevel=False, conf=None, drop_std=False)
    return (
        out['mean'].dropna(how='all'),
        out['std'].dropna(how='all')
    )



In [570]:
# stats
dfs_stats = {}
group_cols = ['lnz_0','phase','temp','beta','volume','system']
dfs_stats['table'], dfs_stats['table_stderr'] = get_stats(dfs['table'], gcol=group_cols)

# spinodal/binodal
# here we need to drop lnz
group_cols_stab = group_cols[1:]
dfs_stats['spin'], dfs_stats['spin_stderr'] = get_stats(dfs['spin'],
                                             gcol=group_cols_stab + ['spinodal','efac'])

dfs_stats['bino'], dfs_stats['bino_stderr'] = get_stats(dfs['bino'],
                                             gcol=group_cols_stab + ['binodal'])

In [588]:
p = Path('../data/NIST_SRSW_Data/LJ_PURE/LJ_3sigma_LRC_TMMC/'.replace('NIST_SRSW_Data','processed'))
p.mkdir(parents=True, exist_ok=True)

for x in ['spin','bino']:
    (
        pd.merge(dfs_stats[x], dfs_stats[x], 
                 left_index=True, right_index=True, suffixes=['','_stderr'])
        .reset_index()
        .to_csv(p / f'{x}-ave.csv', index=False)
    )
